# Webscraping Traditional NBA Pipeline

In [2]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup as bs
from time import sleep
import requests
import pandas as pd
import numpy as np

In [2]:
# Make a url list
url_base = 'https://www.nba.com/stats/players/traditional/?PerMode=Totals&sort=PTS&dir=-1&Season='
url_list = []
a = 2021

while a >= 1996:
    b = a+1
    new_url = url_base + str(a) + "-" + str(b)[-2:] + "&SeasonType=Regular%20Season"
    a -=1
    url_list.append(new_url)

In [3]:
url_list[:5]

['https://www.nba.com/stats/players/traditional/?PerMode=Totals&sort=PTS&dir=-1&Season=2021-22&SeasonType=Regular%20Season',
 'https://www.nba.com/stats/players/traditional/?PerMode=Totals&sort=PTS&dir=-1&Season=2020-21&SeasonType=Regular%20Season',
 'https://www.nba.com/stats/players/traditional/?PerMode=Totals&sort=PTS&dir=-1&Season=2019-20&SeasonType=Regular%20Season',
 'https://www.nba.com/stats/players/traditional/?PerMode=Totals&sort=PTS&dir=-1&Season=2018-19&SeasonType=Regular%20Season',
 'https://www.nba.com/stats/players/traditional/?PerMode=Totals&sort=PTS&dir=-1&Season=2017-18&SeasonType=Regular%20Season']

In [4]:
dataframe_list = []
n=1 # Make a counter to see how many webpages are scraped

# Scrape stat table for each url in url_list
for url in url_list:
    # Use Selenium, provide path to the Edge driver
    s=Service("C:\\Users\sandr\OneDrive\Desktop\Drivers\msedgedriver.exe")
    driver = webdriver.Edge(service=s)
    
    # Make python pause for some time
    sleep(2)

    # Get the page HTML
    try:
        driver.get(url)
        # Make python pause for some time
        sleep(3)
    except WebDriverException:
        print("Webdriver is acting immediately after ‘closing’ the browser")

    # Use Selenium, click on accept cookies button
    driver.find_element(By.XPATH, "/html/body/div[5]/div[2]/div/div[1]/div/div[2]/div/button").click()

    # Make python pause for some time
    sleep(4)

    # Use Selenium, select table page
    select_page = Select(driver.find_element(By.XPATH,"/html/body/main/div/div/div[2]/div/div/nba-stat-table/div[1]/div/div/select"))
    select_page.select_by_visible_text('All')

    # Use BeautifulSoup and lxml extension to parse, save in variable nba_soup
    nba_soup = bs(driver.page_source, 'lxml')
    
    # Locate the table on the page
    table = nba_soup.find('table')
   
    # Find all the headers in the columns
    headers=[]
    for i in table.find_all('th'):
        title = i.text.replace("\n", "").strip()
        headers.append(title)
    
    # Remove redundant headers
    headers_2 = ['PLAYER','TEAM','AGE','GP','W','L','MIN','PTS','FGM','FGA',
                 'FG%','3PM','3PA','3P%','FTM','FTA','FT%','OREB','DREB', 
                 'REB','AST','TOV','STL','BLK','PF','FP','DD2','TD3','+/-','SEASON']

    # Create a dataframe using the headers as column names
    nba_stat = pd.DataFrame(columns = headers_2) 

    # Create a for loop to fill nba_stat
    for i in table.find_all('tr')[1:]:
        row_data = i.find_all('td')[1:]
        row =[j.text.replace("\n", "") for j in row_data] 
        get_season = url.split('Season=')[1] # get the season year
        get_season = get_season.split('&SeasonType')[0] # get the season year
        row.append(get_season) # add the season to the row column
        length = len(nba_stat)
        nba_stat.loc[length] = row
    
    print("Completed:", n, "webpage(s) scraped.")
    n +=1 # add 1 to counter every successful page retrieved
        
    # Close all browsers
    driver.quit()

    # Append table to list of dataframes
    dataframe_list.append(nba_stat)

Webdriver is acting immediately after ‘closing’ the browser
Completed: 1 webpage(s) scraped.
Completed: 2 webpage(s) scraped.
Completed: 3 webpage(s) scraped.
Completed: 4 webpage(s) scraped.
Completed: 5 webpage(s) scraped.
Completed: 6 webpage(s) scraped.
Completed: 7 webpage(s) scraped.
Completed: 8 webpage(s) scraped.
Webdriver is acting immediately after ‘closing’ the browser
Completed: 9 webpage(s) scraped.
Completed: 10 webpage(s) scraped.
Completed: 11 webpage(s) scraped.
Completed: 12 webpage(s) scraped.
Completed: 13 webpage(s) scraped.
Completed: 14 webpage(s) scraped.
Completed: 15 webpage(s) scraped.
Completed: 16 webpage(s) scraped.
Completed: 17 webpage(s) scraped.
Completed: 18 webpage(s) scraped.
Completed: 19 webpage(s) scraped.
Completed: 20 webpage(s) scraped.
Completed: 21 webpage(s) scraped.
Completed: 22 webpage(s) scraped.
Completed: 23 webpage(s) scraped.
Completed: 24 webpage(s) scraped.
Completed: 25 webpage(s) scraped.
Completed: 26 webpage(s) scraped.


In [5]:
# dataframe_list is a list of all dataframes
dataframe_list

[             PLAYER TEAM AGE  GP   W   L   MIN   PTS              FGM  \
 0        Trae Young  ATL  23  76  40  36  2652  2155              711   
 1     DeMar DeRozan  CHI  32  76  43  33  2743  2118              774   
 2       Joel Embiid  PHI  28  68  45  23  2296  2079              666   
 3      Jayson Tatum  BOS  24  76  49  27  2731  2046              708   
 4      Nikola Jokic  DEN  27  74  46  28  2476  2004              764   
 ..              ...  ...  ..  ..  ..  ..   ...   ...              ...   
 600     Nate Hinton  IND  23   2   0   2     2     0                0   
 601      Sam Dekker  TOR  28   1   1   0     1     0                0   
 602  Trayvon Palmer  DET  27   1   0   1    17     0                0   
 603      Tyler Hall  NYK  25   1   1   0     2     0                0   
 604  Zylan Cheatham  UTA  26   1   0   1     5     0                0   
 
                   FGA  ...              AST              TOV              STL  \
 0                1544  ... 

In [6]:
# Concat all the dataframes into one table
nba_stats = pd.concat(dataframe_list, ignore_index=True)
nba_stats.head()

,PLAYER,TEAM,AGE,GP,W,L,MIN,PTS,FGM,FGA,...,AST,TOV,STL,BLK,PF,FP,DD2,TD3,+/-,SEASON
0,Trae Young,ATL,23,76,40,36,2652,2155,711,1544,...,737,303,72,7,128,3535.3,42,0,159,2021-22
1,DeMar DeRozan,CHI,32,76,43,33,2743,2118,774,1535,...,374,181,68,24,178,3244.4,6,0,77,2021-22
2,Joel Embiid,PHI,28,68,45,23,2296,2079,666,1334,...,284,214,77,99,181,3774.2,46,2,368,2021-22
3,Jayson Tatum,BOS,24,76,49,27,2731,2046,708,1564,...,334,217,75,49,174,3432.8,22,0,667,2021-22
4,Nikola Jokic,DEN,27,74,46,28,2476,2004,764,1311,...,584,281,109,63,191,4337.8,66,19,444,2021-22


# Prepare and Clean Data

In [7]:
# Check datatypes
nba_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12307 entries, 0 to 12306
Data columns (total 30 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   PLAYER  12307 non-null  object
 1   TEAM    12307 non-null  object
 2   AGE     12307 non-null  object
 3   GP      12307 non-null  object
 4   W       12307 non-null  object
 5   L       12307 non-null  object
 6   MIN     12307 non-null  object
 7   PTS     12307 non-null  object
 8   FGM     12307 non-null  object
 9   FGA     12307 non-null  object
 10  FG%     12307 non-null  object
 11  3PM     12307 non-null  object
 12  3PA     12307 non-null  object
 13  3P%     12307 non-null  object
 14  FTM     12307 non-null  object
 15  FTA     12307 non-null  object
 16  FT%     12307 non-null  object
 17  OREB    12307 non-null  object
 18  DREB    12307 non-null  object
 19  REB     12307 non-null  object
 20  AST     12307 non-null  object
 21  TOV     12307 non-null  object
 22  STL     12307 non-null

In [8]:
# Get column names
nba_stats.columns

Index(['PLAYER', 'TEAM', 'AGE', 'GP', 'W', 'L', 'MIN', 'PTS', 'FGM', 'FGA',
       'FG%', '3PM', '3PA', '3P%', 'FTM', 'FTA', 'FT%', 'OREB', 'DREB', 'REB',
       'AST', 'TOV', 'STL', 'BLK', 'PF', 'FP', 'DD2', 'TD3', '+/-', 'SEASON'],
      dtype='object')

In [9]:
# Change datatypes to numeric for appropriate columns
nba_stats[['AGE', 'GP', 'W', 'L', 'MIN', 'PTS', 'FGM', 'FGA',
                 'FG%', '3PM', '3PA', '3P%', 'FTM', 'FTA', 'FT%', 
                 'OREB', 'DREB', 'REB','AST', 'TOV', 'STL', 'BLK', 
                 'PF', 'FP', 'DD2', 'TD3', '+/-']] = nba_stats[['AGE', 'GP', 'W', 'L', 'MIN', 'PTS', 
                                                                      'FGM', 'FGA','FG%', '3PM', '3PA', '3P%', 
                                                                      'FTM', 'FTA', 'FT%', 'OREB', 'DREB', 'REB',
                                                                      'AST', 'TOV', 'STL', 'BLK', 'PF', 'FP', 'DD2', 
                                                                      'TD3', '+/-']].apply(pd.to_numeric)

In [10]:
# Verify datatype changes
nba_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12307 entries, 0 to 12306
Data columns (total 30 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   PLAYER  12307 non-null  object 
 1   TEAM    12307 non-null  object 
 2   AGE     12307 non-null  int64  
 3   GP      12307 non-null  int64  
 4   W       12307 non-null  int64  
 5   L       12307 non-null  int64  
 6   MIN     12307 non-null  int64  
 7   PTS     12307 non-null  int64  
 8   FGM     12307 non-null  int64  
 9   FGA     12307 non-null  int64  
 10  FG%     12307 non-null  float64
 11  3PM     12307 non-null  int64  
 12  3PA     12307 non-null  int64  
 13  3P%     12307 non-null  float64
 14  FTM     12307 non-null  int64  
 15  FTA     12307 non-null  int64  
 16  FT%     12307 non-null  float64
 17  OREB    12307 non-null  int64  
 18  DREB    12307 non-null  int64  
 19  REB     12307 non-null  int64  
 20  AST     12307 non-null  int64  
 21  TOV     12307 non-null  int64  
 22

In [11]:
# Look at dimension
nba_stats.shape

(12307, 30)

In [12]:
# Find duplicate rows, if any
duplicateRows = nba_stats[nba_stats.duplicated()]
duplicateRows

,PLAYER,TEAM,AGE,GP,W,L,MIN,PTS,FGM,FGA,...,AST,TOV,STL,BLK,PF,FP,DD2,TD3,+/-,SEASON


There are no duplicate rows

In [32]:
# Find null values, if any
null_counts = nba_stats.isnull().sum()
null_counts

PLAYER    0
AGE       0
GP        0
W         0
L         0
MIN       0
PTS       0
FGM       0
FGA       0
FG%       0
3PM       0
3PA       0
3P%       0
FTM       0
FTA       0
FT%       0
OREB      0
DREB      0
REB       0
AST       0
TOV       0
STL       0
BLK       0
PF        0
DD2       0
TD3       0
+/-       0
SEASON    0
dtype: int64

### Save file

In [15]:
# Save as csv file
nba_stats.to_csv('nba_stats.csv', encoding='utf-8', index=False)

In [16]:
# Check that it's saved
nba_stats = pd.read_csv('nba_stats.csv', encoding='utf-8')
nba_stats

,PLAYER,AGE,GP,W,L,MIN,PTS,FGM,FGA,FG%,...,REB,AST,TOV,STL,BLK,PF,DD2,TD3,+/-,SEASON
0,Trae Young,23,76,40,36,2652,2155,711,1544,46.0,...,284,737,303,72,7,128,42,0,159,2021-22
1,DeMar DeRozan,32,76,43,33,2743,2118,774,1535,50.4,...,392,374,181,68,24,178,6,0,77,2021-22
2,Joel Embiid,28,68,45,23,2296,2079,666,1334,49.9,...,796,284,214,77,99,181,46,2,368,2021-22
3,Jayson Tatum,24,76,49,27,2731,2046,708,1564,45.3,...,609,334,217,75,49,174,22,0,667,2021-22
4,Nikola Jokic,27,74,46,28,2476,2004,764,1311,58.3,...,1019,584,281,109,63,191,66,19,444,2021-22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12302,Anthony Miller,25,1,0,1,14,0,0,5,0.0,...,7,0,0,0,0,2,0,0,-14,1996-97
12303,Bruce Bowen,26,1,1,0,1,0,0,0,0.0,...,0,0,0,0,1,0,0,0,3,1996-97
12304,Cuonzo Martin,25,3,0,3,13,0,0,7,0.0,...,1,1,1,0,0,1,0,0,-2,1996-97
12305,Derrick Alston,24,2,1,1,11,0,0,5,0.0,...,4,0,0,0,0,0,0,0,-7,1996-97


In [13]:
# Remove TEAM and FP (Fantasy Points) columns - irrelevant for this project
nba_stats2 = nba_stats.drop(['TEAM','FP'], axis=1)

nba_stats2.head()

,PLAYER,AGE,GP,W,L,MIN,PTS,FGM,FGA,FG%,...,REB,AST,TOV,STL,BLK,PF,DD2,TD3,+/-,SEASON
0,Trae Young,23,76,40,36,2652,2155,711,1544,46.0,...,284,737,303,72,7,128,42,0,159,2021-22
1,DeMar DeRozan,32,76,43,33,2743,2118,774,1535,50.4,...,392,374,181,68,24,178,6,0,77,2021-22
2,Joel Embiid,28,68,45,23,2296,2079,666,1334,49.9,...,796,284,214,77,99,181,46,2,368,2021-22
3,Jayson Tatum,24,76,49,27,2731,2046,708,1564,45.3,...,609,334,217,75,49,174,22,0,667,2021-22
4,Nikola Jokic,27,74,46,28,2476,2004,764,1311,58.3,...,1019,584,281,109,63,191,66,19,444,2021-22


In [14]:
# Look at dimension
nba_stats2.shape

(12307, 28)

In [5]:
test = nba_stats[nba_stats['PLAYER'] == 'Stephen Curry']
test

,PLAYER,AGE,GP,W,L,MIN,PTS,FGM,FGA,FG%,...,REB,AST,TOV,STL,BLK,PF,DD2,TD3,+/-,SEASON
13,Stephen Curry,34,64,45,19,2211,1630,535,1224,43.7,...,335,404,206,85,23,130,12,2,509,2021-22
605,Stephen Curry,33,63,37,26,2152,2015,658,1365,48.2,...,345,363,213,77,8,119,8,0,244,2020-21
1529,Stephen Curry,32,5,1,4,139,104,33,82,40.2,...,26,33,16,5,2,11,1,0,-45,2019-20
1681,Stephen Curry,31,69,52,17,2331,1881,632,1340,47.2,...,369,361,192,92,25,166,3,0,689,2018-19
2233,Stephen Curry,30,51,41,10,1631,1346,428,864,49.5,...,261,310,153,80,8,114,5,0,486,2017-18
2751,Stephen Curry,29,79,65,14,2639,1999,675,1443,46.8,...,353,524,239,142,17,183,9,0,1015,2016-17
3231,Stephen Curry,28,79,71,8,2700,2375,805,1598,50.4,...,430,527,262,169,15,161,15,2,1022,2015-16
3707,Stephen Curry,27,80,67,13,2613,1900,653,1341,48.7,...,341,619,249,163,16,158,23,0,919,2014-15
4203,Stephen Curry,26,78,50,28,2846,1873,652,1383,47.1,...,334,666,294,128,14,194,28,4,574,2013-14
4686,Stephen Curry,25,78,45,33,2983,1786,626,1388,45.1,...,314,539,240,126,12,198,15,0,133,2012-13


**This tells us that some players play multiple years and will have multiple rows of stats.**

**Some names are mismatched and/or spelled differently, let's fix them.**

In [7]:
nba_stats['PLAYER'] = nba_stats['PLAYER'].str.replace("Tim Hardaway Jr", "Tim Hardaway Jr")

In [11]:
import re
tim = nba_stats[nba_stats['PLAYER'].str.contains('Tim', regex=False)]
tim

,PLAYER,AGE,GP,W,L,MIN,PTS,FGM,FGA,FG%,...,REB,AST,TOV,STL,BLK,PF,DD2,TD3,+/-,SEASON
187,Tim Hardaway Jr.,30,42,24,18,1245,598,209,530,39.4,...,156,92,34,38,6,74,0,0,36,2021-22
343,Timothe Luwawu-Cabarrot,27,52,30,22,685,227,74,186,39.8,...,82,40,20,18,6,76,0,0,-51,2021-22
481,Tim Frazier,31,12,1,11,208,39,14,45,31.1,...,19,34,15,3,1,27,0,0,-80,2021-22
646,Tim Hardaway Jr.,29,70,42,28,1985,1163,407,911,44.7,...,229,126,64,31,11,111,0,0,71,2020-21
856,Timothe Luwawu-Cabarrot,26,58,38,20,1050,370,130,356,36.5,...,129,69,43,33,6,99,0,0,-26,2020-21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11846,Tim Legler,31,8,4,4,76,9,3,19,15.8,...,4,3,4,1,0,11,0,0,0,1997-98
11856,Tim Kempton,34,8,3,5,45,4,2,9,22.2,...,6,3,4,1,2,10,0,0,-15,1997-98
11882,Tim Hardaway,30,81,60,21,3139,1644,575,1384,41.5,...,276,695,230,151,9,165,25,0,540,1996-97
12242,Tim Legler,30,15,10,5,182,44,15,48,31.3,...,21,7,9,3,5,21,0,0,-10,1996-97


In [8]:
nba_stats['PLAYER'] = nba_stats['PLAYER'].str.replace("'",'')

In [9]:
nba_stats['PLAYER'] = nba_stats['PLAYER'].str.replace("Enes Freedom", "Enes Kanter")

In [13]:
nba_stats['PLAYER'] = nba_stats['PLAYER'].str.replace("Otto Porter Jr.", "Otto Porter")

C:\Users\sandr\AppData\Local\Temp/ipykernel_11008/4104035192.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  nba_stats['PLAYER'] = nba_stats['PLAYER'].str.replace("Otto Porter Jr.", "Otto Porter")


In [14]:
otto = nba_stats[nba_stats['PLAYER'].str.contains('Otto', regex=False)]
otto

,PLAYER,AGE,GP,W,L,MIN,PTS,FGM,FGA,FG%,...,REB,AST,TOV,STL,BLK,PF,DD2,TD3,+/-,SEASON
220,Otto Porter,29,63,41,22,1395,515,193,416,46.4,...,362,94,37,68,29,83,5,0,231,2021-22
914,Otto Porter,28,28,12,16,607,271,98,227,43.2,...,152,55,25,16,4,41,2,0,-42,2020-21
1499,Otto Porter,27,14,5,9,331,167,62,140,44.3,...,48,25,11,15,6,31,0,0,26,2019-20
1801,Otto Porter,26,56,25,31,1684,780,299,643,46.5,...,314,120,65,82,31,108,6,0,-22,2018-19
2255,Otto Porter,25,77,42,35,2432,1134,445,885,50.3,...,492,157,75,116,41,151,13,0,286,2017-18
2805,Otto Porter,24,80,48,32,2605,1075,414,803,51.6,...,514,121,44,116,42,193,12,0,191,2016-17
3330,Otto Porter,23,75,36,39,2276,871,339,717,47.3,...,391,120,70,105,32,163,5,0,45,2015-16
3935,Otto Porter,22,74,41,33,1432,445,176,391,45.0,...,221,65,52,44,30,97,0,0,-36,2014-15
4585,Otto Porter,21,37,20,17,319,78,33,91,36.3,...,57,10,14,8,1,26,0,0,-6,2013-14


Each row in this table shows the average stat in a season for each player who has played for at least 5 or more years

## Save into csv file

In [33]:
nba_stats.to_csv('nba_stats_cleaned_final.csv', index=False)

In [34]:
# Check that it's saved
nba_stats = pd.read_csv('nba_stats_cleaned_final.csv')
nba_stats

,PLAYER,AGE,GP,W,L,MIN,PTS,FGM,FGA,FG%,...,REB,AST,TOV,STL,BLK,PF,DD2,TD3,+/-,SEASON
0,Trae Young,23,76,40,36,2652,2155,711,1544,46.0,...,284,737,303,72,7,128,42,0,159,2021-22
1,DeMar DeRozan,32,76,43,33,2743,2118,774,1535,50.4,...,392,374,181,68,24,178,6,0,77,2021-22
2,Joel Embiid,28,68,45,23,2296,2079,666,1334,49.9,...,796,284,214,77,99,181,46,2,368,2021-22
3,Jayson Tatum,24,76,49,27,2731,2046,708,1564,45.3,...,609,334,217,75,49,174,22,0,667,2021-22
4,Nikola Jokic,27,74,46,28,2476,2004,764,1311,58.3,...,1019,584,281,109,63,191,66,19,444,2021-22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12302,Anthony Miller,25,1,0,1,14,0,0,5,0.0,...,7,0,0,0,0,2,0,0,-14,1996-97
12303,Bruce Bowen,26,1,1,0,1,0,0,0,0.0,...,0,0,0,0,1,0,0,0,3,1996-97
12304,Cuonzo Martin,25,3,0,3,13,0,0,7,0.0,...,1,1,1,0,0,1,0,0,-2,1996-97
12305,Derrick Alston,24,2,1,1,11,0,0,5,0.0,...,4,0,0,0,0,0,0,0,-7,1996-97
